In [12]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.feature_extraction.dict_vectorizer import DictVectorizer
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.metrics.classification import classification_report, accuracy_score, confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.tree.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

from sklearn.preprocessing import MinMaxScaler

INPUT_DIR = '../input'


In [6]:
# Define function to extract titles from passenger names
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    # If the title exists, extract and return it.
    if title_search:
        return title_search.group(1)
    return ""


def min_max_scale(train_data, test_data, cat_cols):
    
    data = pd.concat([train_data, test_data])
    
    # numeric attributes
    num_data = data.drop(cat_cols, axis=1)
    
    # fit scaler on all data
    scaler = MinMaxScaler().fit(num_data)
    
    # transform all data with scaler
    train_data = scaler.transform(train_data.drop(cat_cols, axis=1))
    test_data = scaler.transform(test_data.drop(cat_cols, axis=1))
    
    # scale to <0,1>
    num_train_data = pd.DataFrame(train_data)
    num_test_data = pd.DataFrame(test_data)

    # fill nan with mean column values
    num_train_data.fillna(data.mean(), inplace=True)
    num_test_data.fillna(data.mean(), inplace=True)

    return num_train_data, num_test_data


def cat_vectorize(train_data, test_data, num_cols):
    # categorical attributes
    cat_train_data = train_data.drop(num_cols, axis=1)
    cat_test_data = test_data.drop(num_cols, axis=1)

    cat_train_data.fillna('NA', inplace=True)
    cat_test_data.fillna('NA', inplace=True)

    cat_train_data_values = cat_train_data.T.to_dict().values()
    cat_test_data_values = cat_test_data.T.to_dict().values()

    # vectorize (encode as one hot)
    vectorizer = DictVectorizer(sparse=False)
    vec_train_data = vectorizer.fit_transform(cat_train_data_values)
    vec_test_data = vectorizer.transform(cat_test_data_values)

    return vec_train_data, vec_test_data


In [7]:
""" -------------------------------------- Data loading ---------------------------------------- """

# load dataframes
df_train = pd.read_csv(os.path.join(INPUT_DIR, 'train.csv'))
df_test = pd.read_csv(os.path.join(INPUT_DIR, 'test.csv'))

df_full = [df_train, df_test]

print(df_train.head())


   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  


In [8]:
""" -------------------------------------- Feature Engineering ---------------------------------------- """

for dataset in df_full:
    dataset['Name_length'] = dataset['Name'].apply(len)

    # Feature that tells whether a passenger had a cabin on the Titanic
    dataset['Has_Cabin'] = dataset["Cabin"].apply(lambda x: 0 if type(x) == float else 1)

    # Create new feature FamilySize as a combination of SibSp and Parch
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

    # Create new feature IsAlone from FamilySize
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1

    # Remove all NULLS in the Embarked column
    dataset['Embarked'] = dataset['Embarked'].fillna('S')

    # Remove all NULLS in the Fare column and create a new feature CategoricalFare
    dataset['Fare'] = dataset['Fare'].fillna(df_train['Fare'].median())

    # df_train['CategoricalFare'] = pandas.qcut(df_train['Fare'], 4)

    # Create a New feature CategoricalAge
    age_avg = dataset['Age'].mean()
    age_std = dataset['Age'].std()
    age_null_count = dataset['Age'].isnull().sum()
    age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size=age_null_count)
    dataset['Age'][np.isnan(dataset['Age'])] = age_null_random_list
    dataset['Age'] = dataset['Age'].astype(int)

    # df_train['CategoricalAge'] = pandas.cut(df_train['Age'], 5)

    # Create a new feature Title, containing the titles of passenger names
    dataset['Title'] = dataset['Name'].apply(get_title)

    # Group all non-common titles into one single grouping "Rare"
    dataset['Title'] = dataset['Title'].replace(
        ['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

    # Mapping Sex
    dataset['Sex'] = dataset['Sex'].map({'female': 0, 'male': 1}).astype(int)

    # Mapping titles
    title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

    # Mapping Embarked
    dataset['Embarked'] = dataset['Embarked'].map({'S': 0, 'C': 1, 'Q': 2}).astype(int)

    # Mapping Fare
    dataset.loc[dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare'] = 2
    dataset.loc[dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

    # Mapping Age
    dataset.loc[dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[dataset['Age'] > 64, 'Age'] = 4

print(df_train.head(10))

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   
5            6         0       3   
6            7         0       1   
7            8         0       3   
8            9         1       3   
9           10         1       2   

                                                Name  Sex  Age  SibSp  Parch  \
0                            Braund, Mr. Owen Harris    1    1      1      0   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...    0    2      1      0   
2                             Heikkinen, Miss. Laina    0    1      0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)    0    2      1      0   
4                           Allen, Mr. William Henry    1    2      0      0   
5                                   Moran, Mr. James    1    1      0      0   
6                            McCarthy, Mr. 

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [9]:
""" -------------------------------------- Feature preparation ---------------------------------------- """

label_column = 'Survived'

# get all column names
cols = list(df_train.columns.values)

# numeric columns
num_cols = [e for e in df_train.select_dtypes(include=[np.number]).columns.tolist() if e != label_column]

# categorical columns
cat_cols = [e for e in cols if e not in num_cols and e != label_column]

print(num_cols, cat_cols)

x_train, y_train = df_train.drop(label_column, axis=1), df_train[label_column]
x_test = df_test

# scale everything to [0, 1]
x_num_train, x_num_test = min_max_scale(x_train, x_test, cat_cols)

# vectorize categorical columns
vec_x_cat_train, vec_x_cat_test = cat_vectorize(x_train, x_test, num_cols)

# build the feature vector
x_train = np.hstack((x_num_train, vec_x_cat_train))
x_test = np.hstack((x_num_test, vec_x_cat_test))

# labels or target attribute
y_train = y_train.astype(int)

['PassengerId', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Name_length', 'Has_Cabin', 'FamilySize', 'IsAlone', 'Title'] ['Name', 'Ticket', 'Cabin']


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [ ]:
""" -------------------------------------- Cross validation ---------------------------------------- """

# split the data into train and test
xs_train, xs_test, ys_train, ys_test = train_test_split(x_train, y_train, test_size=0.2)


In [ ]:
""" -------------------------------------- Best Linear Regression search ---------------------------------------- """

# Create the parameter grid 
gbm_param_grid = {
    'n_estimators': range(8, 20),
    'max_depth': range(6, 10),
    'learning_rate': [.4, .45, .5, .55, .6],
    'colsample_bytree': [.6, .7, .8, .9, 1]
}

In [11]:
""" -------------------------------------- Best params search ---------------------------------------- """

# Create the parameter grid 
gbm_param_grid = {
    'n_estimators': range(8, 20),
    'max_depth': range(6, 10),
    'learning_rate': [.4, .45, .5, .55, .6],
    'colsample_bytree': [.6, .7, .8, .9, 1]
}

# Instantiate the regressor: gbm
gbm = XGBClassifier(n_estimators=10)

# Perform random search: grid_mse
xgb_random = RandomizedSearchCV(param_distributions=gbm_param_grid,
                                estimator=gbm, scoring="accuracy",
                                verbose=1, n_iter=50, cv=4)

# Fit randomized_mse to the data
xgb_random.fit(x_train, y_train)

# Print the best parameters and lowest RMSE
print("Best parameters found: ", xgb_random.best_params_)
print("Best accuracy found: ", xgb_random.best_score_)


Fitting 4 folds for each of 50 candidates, totalling 200 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:  2.5min finished


Best parameters found:  {'n_estimators': 15, 'max_depth': 8, 'learning_rate': 0.5, 'colsample_bytree': 0.6}
Best accuracy found:  0.8258426966292135


In [ ]:
pred = xgb_random.predict(x_test)
submission = pd.DataFrame({'PassengerId': df_test['PassengerId'], 'Survived': pred})
submission.to_csv('gender_submission.csv', index=False)